<a href="https://colab.research.google.com/github/mkbahk/graphcore-ipu-demo/blob/main/docker_jupyter_mnist_keras_sample_20201015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[도커내의 jupyter notebook접근하는 방법]


**$docker inspect f21be | grep IPAddress**
            "SecondaryIPAddresses": null,
            "IPAddress": "172.17.0.3",
                    "IPAddress": "172.17.0.3",

**$sudo iptables -A DOCKER -p tcp --dport 8888 -j ACCEPT -d 172.17.0.3**

**$sudo iptables -t nat -nL**

Chain DOCKER (1 references)
target     prot opt source               destination         
ACCEPT     tcp  --  0.0.0.0/0            172.17.0.3           tcp dpt:8888


**$ sudo iptables -t nat -A DOCKER -p tcp --dport 8888 -j DNAT --to 172.17.0.3:8888**

**$ sudo iptables -t nat -A POSTROUTING -p tcp --dport 8888 -j MASQUERADE -s 172.17.0.3 -d 172.17.0.3**


Chain POSTROUTING (policy ACCEPT)
target     prot opt source               destination         
MASQUERADE  all  --  172.17.0.0/16        0.0.0.0/0           
MASQUERADE  tcp  --  172.17.0.3           172.17.0.3           tcp dpt:8888
Chain DOCKER (2 references)
target     prot opt source               destination         
RETURN     all  --  0.0.0.0/0            0.0.0.0/0           
DNAT       tcp  --  0.0.0.0/0            0.0.0.0/0            tcp dpt:8888 to:172.17.0.3:8888



[도커안에서 jupyter-notebook실행]

**root@bad3da9c3d34:/# jupyter notebook --ip=0.0.0.0 --port=8888 --allow-root &   **

    
    To access the notebook, open this file in a browser:
        file:///root/.local/share/jupyter/runtime/nbserver-25-open.html
    Or copy and paste one of these URLs:
        http://bad3da9c3d34:9999/?token=6ad47b1335106ca93bf28e7a2916925ad10ffa886cfaf67e
     or http://127.0.0.1:9999/?token=6ad47b1335106ca93bf28e7a2916925ad10ffa886cfaf67e
     


[브라우저에서 접속하는 방법]


**http://호스트ip:9999/?Token=6ad47b1335106ca93bf28e7a2916925ad10ffa886cfaf67e**


In [ ]:
# Module Import
from tensorflow import keras

In [ ]:
# Load MNIST DataSet
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
# one-hot enconding 수행
# 5 --> 0 0 0 0 0 1 0 0 0 0
# 1 --> 0 1 0 0 0 0 0 0 0 0
y_train = keras.utils.to_categorical(y=y_train, num_classes = 10)
y_test = keras.utils.to_categorical(y=y_test, num_classes=10)

In [ ]:
#Reshaping DataSet
x_train = x_train.reshape(60000, 28*28)
x_test = x_test.reshape(10000, 28*28)
print(y_train.shape, y_test.shape)
print(x_train.shape, x_test.shape)

In [ ]:
#모델생성
model = keras.Sequential()
model.add(keras.layers.Dense(32, activation="sigmoid", input_shape=(28*28,)))
model.add(keras.layers.Dense(32, activation="sigmoid"))
model.add(keras.layers.Dense(10, activation="sigmoid"))

In [ ]:
#모델컴파일
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.1), loss="categorical_crossentropy", metrics=['accuracy'])
model.summary()

In [ ]:
#모델훈련
model.fit(x=x_train, y=y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

In [ ]:
#모델 평가
model.evaluate(x_test, y_test )

END OF CODE